In [ ]:
###CONFIGURAÇÕES INICIAIS###

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, monotonically_increasing_id, when, date_format
from datetime import datetime, timedelta
import holidays

# Inicializando a sessão do Spark
spark = SparkSession.builder \
    .appName("Bronze") \
    .getOrCreate()

# Caminho para salvar o arquivo no Google Drive
file_path = r'C:\Users\yurib\pjf-engenharia-de-dados\algo'

In [ ]:
### CARREGANDO ARQUIVOS DA LANDING ###

# Carregar dados da camada landing para Spark
pedidos = spark.read.csv(f"{file_path}/landing/pedidos.csv", header=True, inferSchema=True)
clientes = spark.read.csv(f"{file_path}/landing/clientes.csv", header=True, inferSchema=True)
vendedores = spark.read.csv(f"{file_path}/landing/vendedores.csv", header=True, inferSchema=True)
pagamentos = spark.read.csv(f"{file_path}/landing/pagamentos.csv", header=True, inferSchema=True)
dadospedidos = spark.read.csv(f"{file_path}/landing/dadospedidos.csv", header=True, inferSchema=True)

In [26]:
### CRIAÇÃO DA DIM_DATA ###

# Gerar uma lista de datas de 2015 até hoje + 1 dia
end_date = datetime.now() + timedelta(days=1)
dates = [(datetime(2015, 1, 1) + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end_date - datetime(2015, 1, 1)).days + 1)]

# Criar DataFrame com Spark
df = spark.createDataFrame([(date,) for date in dates], ["dt_data"])

# Obter feriados no Brasil
br_holidays = holidays.Brazil(years=range(2015, end_date.year + 1))
br_holidays_set = set([str(date) for date in br_holidays])

# Adicionar uma coluna indicando se a data é útil (não é feriado e não é fim de semana)
df = df.withColumn('dt_util', (~col('dt_data').isin([lit(holiday) for holiday in br_holidays_set])) & (~dayofweek(col('dt_data')).isin([7, 1])))


 Adicionar colunas de dia da semana, mês/ano, trimestre e ano
df = df.withColumn('dt_diasemana', date_format('dt_data', 'EEEE')) \
       .withColumn('dt_diasemana_ord', dayofweek(col('dt_data'))) \
       .withColumn('dt_mesano', date_format('dt_data', 'MMMM-yyyy')) \
       .withColumn('dt_mesano_ord', (year('dt_data') - 2015) * 12 + month('dt_data')) \
       .withColumn('dt_trimestre', concat(year('dt_data'), lit('-Q'), quarter('dt_data'))) \
       .withColumn('dt_trimestre_ord', (year('dt_data') - 2015) * 4 + quarter('dt_data')) \
       .withColumn('dt_ano', year('dt_data')) \
       .withColumn('sk_data', monotonically_increasing_id())

# Reorganizar as colunas conforme o formato especificado
df = df.select('dt_util', 'dt_data', 'dt_diasemana', 'dt_diasemana_ord', 'dt_mesano', 'dt_mesano_ord', 'dt_trimestre', 'dt_trimestre_ord', 'dt_ano', 'sk_data')


In [27]:
### CRIAÇÃO DE PEDIDOS ###

# Renomear colunas
clientes = clientes.withColumnRenamed('idgeralcliente', 'idcliente')

# Realizar merges dos dados de pedidos
pedidos = pedidos.join(dadospedidos, on='idpedido', how='left') \
                 .join(pagamentos, on='idpedido', how='left') \
                 .join(vendedores, on='idvendedor', how='left') \
                 .join(clientes, on='idcliente', how='left')

# Realizar merge dos produtos com os pedidos
pedidos = pedidos.join(produtos, pedidos['idproduto'] == produtos['id_produto'], how='left')

In [29]:
# Adicionar colunas de chave sequencial
produtos = produtos.withColumn('SK_Produto', monotonically_increasing_id())
localizacoes = localizacoes.withColumn('SK_Localizacoes', monotonically_increasing_id())
pedidos = pedidos.withColumn('SK_Pedidos', monotonically_increasing_id())

In [30]:
### SALVANDO EM SPARK NA CAMADA BRONZE ###

localizacoes.write.mode('overwrite').saveAsTable('localizacoes')
produtos.write.mode('overwrite').saveAsTable('produtos')
pedidos.write.mode('overwrite').saveAsTable('pedidos')
df.write.mode('overwrite').saveAsTable('dim_data')

In [ ]:
# Fim da sessão Spark
spark.stop()